# Fixations to Bottom-Strip
Determine the value of hyperparameter `cnfg.FIXATIONS_TO_STRIP_THRESHOLD`, which is used to decide if a fixation / visit is a _LWS_ instance or not by excluding _LWS_ candidates that occur too close to a fixation in the bottom-strip (where exemplars are presented).

##### Update:
There's not much difference if this value is set to 2/3/4 fixations, so we can choose 3 fixations as a reasonable compromise.

In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import config as cnfg
from data_models.SearchArray import SearchArray

pio.renderers.default = "notebook"  # or "browser"

### Read data

In [4]:
from preprocess.read_data import read_saved_data
_targets, actions, _metadata, idents, fixations, _visits = read_saved_data(cnfg.OUTPUT_PATH)

In [5]:
fixs_with_ident_time = fixations.copy()
dva_cols = [col for col in fixations.columns if col.endswith("distance_dva")]
fixs_with_ident_time["target"] = fixs_with_ident_time[dva_cols].idxmin(axis=1).str.replace("_distance_dva", "")
fixs_with_ident_time = (
    fixs_with_ident_time
    .drop(columns=[col for col in fixs_with_ident_time.columns if "_distance_" in col])
    .merge(
        idents.loc[
            idents["identification_category"] == "hit", ["subject", "trial", "target", "time"]
        ], on=["subject", "trial", "target"], how="left"
    )
)

fixs_with_ident_time.loc[:, "is_during"] = (fixs_with_ident_time["start_time"] <= fixs_with_ident_time["time"]) & (fixs_with_ident_time["time"] <= fixs_with_ident_time["end_time"])

fixs_with_ident_time.loc[:, "is_in_strip"] = fixs_with_ident_time.apply(
    lambda row: SearchArray.is_in_bottom_strip(tuple([row["x"], row["y"]])), axis=1
)

##### Count how many fixations are between a bottom-strip fixation and a target-identification fixation:

In [6]:
def count_fixations_from_strip_to_identification(is_in_strip: pd.Series, is_during: pd.Series) -> pd.Series:
    """
    For a single trial's fixations, count how many fixations are between a bottom-strip fixation and a target-identification fixation.
    For non-bottom-strip fixations, the count is NaN.
    """
    assert len(is_in_strip) == len(is_during), "is_in_strip and is_during must have the same length"
    assert is_in_strip.index.equals(is_during.index), "is_in_strip and is_during must have the same index"
    num_fixs_between = pd.Series(np.nan, index=is_during.index, dtype=float)
    during_indices = np.flatnonzero(is_during.values)
    if len(during_indices) == 0:
        return num_fixs_between     # no identification fixations - return NaN for all fixations
    row_indices = np.arange(len(is_during))
    next_during_indices = np.searchsorted(during_indices, row_indices, side="right")
    has_subsequent_ident = next_during_indices < len(during_indices)
    ident_distances = np.full_like(is_during, np.nan, dtype=float)
    ident_distances[has_subsequent_ident] = during_indices[next_during_indices[has_subsequent_ident]] - row_indices[has_subsequent_ident]    # distance between each fixation and the next identification fixation
    num_fixs_between[is_in_strip] = ident_distances[is_in_strip]    # apply only to bottom-strip fixations
    return num_fixs_between


# populate the dataframe
fixs_with_ident_time["num_fixs_strip_to_ident"] = np.nan
for (subj_id, trial_num, eye), data in fixs_with_ident_time.groupby(["subject", "trial", "eye"]):
    data = data.sort_values("start_time")
    num_fixs_to_strip = count_fixations_from_strip_to_identification(
        data["is_in_strip"], data["is_during"]
    )
    fixs_with_ident_time.loc[data.index, "num_fixs_strip_to_ident"] = num_fixs_to_strip

### Number of Fixations until Bottom-Strip Visit
#### (1) All Fixations

In [7]:
valid_fixs_with_ident_time = fixs_with_ident_time.loc[np.isfinite(fixs_with_ident_time["num_fixs_to_strip"])]

fixs_summary = (
    pd.concat([
        valid_fixs_with_ident_time["num_fixs_to_strip"].describe().rename("all"),
        valid_fixs_with_ident_time.groupby("subject")["num_fixs_to_strip"].describe().T,
    ], axis=1)
).T

print("All Fixations:")
fixs_summary

All Fixations:


,count,mean,std,min,25%,50%,75%,max
all,80401.0,22.930959,21.932342,0.0,6.0,16.0,33.0,158.0
2,8223.0,33.758482,28.520935,0.0,11.0,26.0,50.0,158.0
12,8311.0,24.176874,23.230286,0.0,7.0,17.0,35.0,124.0
13,6451.0,17.481321,15.844743,0.0,5.0,13.0,26.0,101.0
14,6597.0,21.163408,20.793083,0.0,5.0,15.0,31.0,140.0
15,5884.0,17.472638,17.615970,0.0,5.0,12.0,25.0,98.0
16,7509.0,21.466773,19.041437,0.0,7.0,16.0,31.0,88.0
17,4105.0,21.873812,21.727067,0.0,6.0,16.0,30.0,146.0
18,6874.0,27.450102,24.482246,0.0,8.0,20.0,42.0,100.0
19,6053.0,13.543367,13.492378,0.0,4.0,9.0,19.0,77.0


#### (2) Ident Fixations

In [8]:
ident_fixs = fixs_with_ident_time.loc[fixs_with_ident_time["is_during"]]
valid_ident_fixs_with_ident_time = ident_fixs.loc[np.isfinite(fixs_with_ident_time["num_fixs_to_strip"])]

fixs_summary = (
    pd.concat([
        valid_ident_fixs_with_ident_time["num_fixs_to_strip"].describe().rename("all"),
        valid_ident_fixs_with_ident_time.groupby("subject")["num_fixs_to_strip"].describe().T,
    ], axis=1)
).T

print("Identification Fixations:")
fixs_summary

Identification Fixations:


,count,mean,std,min,25%,50%,75%,max
all,1433.0,19.936497,21.589738,1.0,4.0,12.0,28.00,126.0
2,111.0,31.675676,29.260324,1.0,9.0,24.0,43.50,126.0
12,104.0,22.750000,22.773153,1.0,5.0,15.0,31.25,101.0
13,139.0,17.913669,17.816082,1.0,4.0,12.0,25.00,100.0
14,137.0,15.000000,18.589608,1.0,2.0,9.0,19.00,101.0
15,118.0,10.550847,14.434999,1.0,2.0,5.0,13.75,74.0
16,138.0,23.108696,19.446608,1.0,11.0,16.5,30.75,79.0
17,74.0,18.094595,15.368683,1.0,7.0,13.0,25.75,57.0
18,145.0,24.103448,23.466798,1.0,6.0,14.0,39.00,97.0
19,138.0,10.434783,11.945715,1.0,2.0,6.0,13.75,63.0


### Number of Fixations from Bottom-Strip to Identification

In [9]:
finite_num_from_strip_to_iden = fixs_with_ident_time.loc[np.isfinite(fixs_with_ident_time["num_fixs_strip_to_ident"])]

percentiles = [0.01, 0.05, 0.1, 0.25, 0.5]
fixs_summary = (
    pd.concat([
        finite_num_from_strip_to_iden["num_fixs_strip_to_ident"].describe(percentiles).rename("all"),
        finite_num_from_strip_to_iden.groupby("subject")["num_fixs_strip_to_ident"].describe(percentiles).T,
    ], axis=1)
).T

print("From Strip to Identification:")
fixs_summary

From Strip to Identification:


,count,mean,std,min,1%,5%,10%,25%,50%,max
all,2402.0,19.579517,16.953980,1.0,1.00,2.0,3.0,6.0,15.0,111.0
2,120.0,20.391667,16.744896,1.0,1.00,2.0,3.0,6.0,15.5,68.0
12,207.0,24.207729,23.159647,1.0,2.00,3.0,4.0,7.0,15.0,111.0
13,222.0,14.626126,13.280811,2.0,2.00,2.0,2.0,3.0,10.0,65.0
14,192.0,20.156250,13.997627,1.0,1.91,2.0,4.0,10.0,17.0,63.0
15,273.0,18.571429,16.167421,1.0,1.00,2.0,3.0,6.0,13.0,70.0
16,237.0,20.590717,17.414111,1.0,1.00,2.0,3.0,6.0,16.0,70.0
17,111.0,15.135135,14.336531,1.0,1.00,2.0,2.0,3.0,10.0,46.0
18,206.0,25.165049,17.704125,1.0,2.00,2.0,3.0,10.0,24.0,71.0
19,228.0,17.614035,14.747214,1.0,1.00,2.0,3.0,6.0,13.0,63.0
